In [1]:
import pandas as pd
fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head() # 데이터 프레임의 첫번째 행부터 5개 추출

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [2]:
# pandas unique : 명시된 컬럼에 중복 제거
print(pd.unique(fish['Species']))

['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']


In [5]:
# Species : 타겟 데이터(정답 데이터), 그외 - 학습 데이터
import numpy as np
fish_input = fish[['Weight', 'Length', 'Diagonal', 'Height', 'Width']].to_numpy()
print(fish_input[:5])

[[242.      25.4     30.      11.52     4.02  ]
 [290.      26.3     31.2     12.48     4.3056]
 [340.      26.5     31.1     12.3778   4.6961]
 [363.      29.      33.5     12.73     4.4555]
 [430.      29.      34.      12.444    5.134 ]]


In [6]:
fish_target = fish['Species'].to_numpy()  # 타겟 데이터(정답 데이터)

In [7]:
from sklearn.model_selection import train_test_split  # 학습 세트, 테스트 세트 분리

train_input, test_input, train_target, test_target = train_test_split(
    fish_input, fish_target
)

In [8]:
# 데이터 전처리 - 표준 점수 변환 
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)  # 훈련 세트 훈련 데이터를 표준 점수로 변환
test_scaled = ss.transform(test_input) # 테스트 세트 데이터를 표준 점수로 변환
print(train_scaled[:5])

[[-0.43640294 -0.10450739 -0.21942844 -0.41551352  0.06656912]
 [ 3.46762594  3.19775616  3.10840273  0.41084745  1.78141367]
 [-0.65949031 -0.47244762 -0.55136045 -0.65148955 -0.42849823]
 [ 1.4458967   0.99931329  1.17638821  2.27051087  1.35161376]
 [ 0.81846349  0.58538054  0.76785651  1.68192887  0.66851263]]


In [9]:
# k-최근접 이웃 분류 - 분류, 확율 예측
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target) # 학습

print(kn.score(train_scaled, train_target)) # 훈련 세트 정확도
print(kn.score(test_scaled, test_target)) # 테스트 세트 정확도

0.8991596638655462
0.725


In [10]:
# 클래스 목록 (분류 항목) - 알파벳 순서대로 
print(kn.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [11]:
print(kn.predict(test_scaled[:5]))

['Perch' 'Perch' 'Perch' 'Pike' 'Perch']


In [13]:
# 분류 기준 확률 kn.predict_proba(..)
proba = kn.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=3))


[[0.    0.    1.    0.    0.    0.    0.   ]
 [0.    0.    0.667 0.    0.333 0.    0.   ]
 [0.    0.    0.667 0.    0.333 0.    0.   ]
 [0.    0.    0.    1.    0.    0.    0.   ]
 [0.    0.    0.667 0.    0.333 0.    0.   ]]


In [18]:
distances, indexes = kn.kneighbors(test_scaled[2:3])
print(train_target[indexes])  # 확률로 보기에는 조금 어색 -> 로지스틱 회귀 

[['Perch' 'Perch' 'Roach']]
